In [11]:
!rm data.zip
!wget https://github.com/Dunckleosteus/seismic/blob/18908e9e23b56529c5744c96fac2a5216cd298cd/data.zip
!unzip data.zip

--2025-03-19 09:17:39--  https://github.com/Dunckleosteus/seismic/blob/18908e9e23b56529c5744c96fac2a5216cd298cd/data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data.zip’

data.zip                [ <=>                ] 203.17K  --.-KB/s    in 0.05s   

2025-03-19 09:17:39 (3.87 MB/s) - ‘data.zip’ saved [208048]

Archive:  data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data.zip or
        data.zip.zip, and cannot find data.zip.ZIP, period.
